**Resumen automático**

Práctica de PLN en la que en base a métodos extractivos se hará un resumen del discurso de Steve Jobs.

Por: Martínez Cadena Gustavo

---

# Por frecuencia de palabras (Term frequency)

La forma más simple de hacer un resumen con el enfoque extractivo es con el: **Resumen basado en la frecuencia de palabras**.

Este método sigue los siguientes pasos:

1.   Limpiar el documento removiendo stopwords, números, signos de puntuación y demás caracteres especiales.
2.   Separar el documento en oraciones.
3.   Contar cuántas veces aparece cada palabra en el documento y dividirlo por el número de veces que aparece la palabra más frecuente en el documento para obtener la *frecuencia de palabras*.
4.    Sumar la frecuencia de palabras de todas las palabras que aparecen en la misma oración y obtener el score de cada oración.
5.    Mantener las oraciones con un score superior a cierto umbral para hacer el resumen.

In [ ]:
import nltk
import re
import string
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def cargar_texto(path: str) -> str:
  with open(path, "r") as archivo:
    texto = archivo.read()
  return texto

In [94]:
def oracion(texto:str) -> list[str]:
  texto_por_oracion = []
  oracion = ""
  for letra in texto:
    if letra == ".":
      texto_por_oracion.append(oracion.strip()) # quita espacio inicial
      oracion = ""
    else:
      oracion += letra

  return texto_por_oracion

In [34]:
def no_caracter_especial(texto: str) -> str:
  limpiar = re.findall(r"[^-_—¡!¿?'.,;:1234567890“”\[\]\"`]", texto) # quita los caracteres especiales con expresión regular
  return ''.join(limpiar) # list[str] -> str

In [ ]:
def preprocesar(texto: str) -> list[str]:
  # normalizar
  texto = texto.lower()
  # quitar caracteres especiales
  texto = no_caracter_especial(texto)
  # tokenizar
  nltk.download('punkt_tab')
  tokens = word_tokenize(texto, language="spanish")
  # stopwords
  nltk.download("stopwords")
  stopwords_esp = set(stopwords.words("spanish"))
  no_stopwords = []

  for palabra in tokens:
    if palabra not in stopwords_esp:
      no_stopwords.append(palabra)
  #stemming
  spanish_stemmer = SnowballStemmer('spanish')
  texto_stemming = []
  for palabra in no_stopwords:
    texto_stemming.append(spanish_stemmer.stem(palabra))

  return texto_stemming

In [ ]:
def frecuencia_palabras(lista_palabras: list[str], frecuencia: dict) -> dict[str, int]:
  for palabra in lista_palabras:
    if palabra in frecuencia:
      frecuencia[palabra] += 1
    else:
      frecuencia[palabra] = 1
  return frecuencia

In [37]:
# Main =================================================
# Cargar txt y dividirlo por oraciones
path = "/content/Discurso.txt"
texto = cargar_texto(path)
dividir_por_oracion = oracion(texto)

# preprocesamiento de texto por oraciones
texto_preprocesado = []
for oracion in dividir_por_oracion:
  texto_preprocesado.append(preprocesar(oracion))

# Contar la frecuencia de cada palabra
diccionario_frecuencias = {}
for sublista_texto in texto_preprocesado:
  diccionario_frecuencias = frecuencia_palabras(sublista_texto, diccionario_frecuencias)

# Obtener la frecuencia máxima y dividir cada palabra para obtener frecuencia de palabras
clave = max(diccionario_frecuencias, key=diccionario_frecuencias.get)
maximo = diccionario_frecuencias[clave]
for x, y in diccionario_frecuencias.items():
  if maximo != 0:
    diccionario_frecuencias[x] = y/maximo # # veces que aparece palabra / # palabra más frecuente

# Puntuar cada oración sumando la frecuencia de palabra de cada una de las palabras que lo conforman
puntuacion_oracion = []
for oracion in texto_preprocesado:
  sumar_puntuacion = 0
  for palabra in oracion:
    sumar_puntuacion += diccionario_frecuencias[palabra]
  puntuacion_oracion.append(sumar_puntuacion)

# Resumen conformado por oraciones con una puntuación mayor a 3
resumen = ""
umbral = 3 # Umbral propuesto
for a, b in zip(puntuacion_oracion, dividir_por_oracion): # Como ambas están en el mismo orden, se recorren simultáneamente
  if a >= umbral:
    resumen += b + ". "

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /ro

In [38]:
resumen

'Dejé la Universidad de Reed tras los seis primeros meses, pero después seguí vagando por allí otros 18 meses, más o menos, antes de dejarlo del todo. No tenía idea de qué quería hacer con mi vida, y menos aún de cómo la universidad me iba a ayudar a averiguarlo. Aprendí cosas sobre el serif y tipografías sans serif, sobre los espacios variables entre letras, sobre qué hace realmente grande a una gran tipografía. Si nunca me hubiera dejado caer por aquél curso concreto en la universidad, el Mac jamás habría tenido múltiples tipografías, ni caracteres con espaciado proporcional. Si nunca hubiera decidido dejarlo, no habría entrado en esa clase de caligrafía y los computadores personales no tendrían la maravillosa tipografía que poseen. Por supuesto, era imposible conectar los puntos mirando hacia el futuro cuando estaba en clase, pero fue muy, muy claro al mirar atrás diez años más tarde. Lo diré otra vez: no puedes conectar los puntos hacia adelante, sólo puedes hacerlo hacia atrás. Tr

**Anexo:** Para determinar el umbral ideal, se optó por graficar la evolución del score de cada oración.


---



In [ ]:
import plotly.express as px
import pandas as pd

# Creamos un DataFrame con índices y valores
df = pd.DataFrame({
    "Índice": list(range(len(puntuacion_oracion))),
    "Valor": puntuacion_oracion
})
# Diagrama de líneas
fig = px.line(df, x="Índice", y="Valor", title="Diagrama de evolución de score de cada oración")
fig.show()

In [39]:
# ¿Cuál es el tamaño de las oraciones aceptadas?
tamanio_oraciones = []
for a, b in zip(puntuacion_oracion, dividir_por_oracion):
  if a >= umbral:
    tamanio_oraciones.append(len(b))

In [40]:
tamanio_oraciones

[148,
 111,
 151,
 166,
 158,
 154,
 95,
 133,
 130,
 233,
 114,
 137,
 124,
 289,
 218,
 81,
 151,
 151,
 206]

# TF - IDF (Term Frequency - Inverse Document Frequency)

Método más riguroso que mide la relevancia de cada término.
Los pasos que se llevaron a cabo para hacer un resumen con el enfoque extractivo son los siguientes:

1. Limpiar el documentos.
2. Separar el texto en oraciones (En términos de TF-IDF su equivalente son los documentos).
3. Dado que no se usan palabras clave, el relevancia de cada oración estuvo en función de la puntuación media que tuvieran.
4. Mantener las oraciones con el mayor puntaje en el orden en que estuvieran en el texto original.


---



In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [95]:
def tf_idf(corpus: list[str]):
  vectorizer = TfidfVectorizer()
  X = vectorizer.fit_transform(corpus)
  return vectorizer.get_feature_names_out(), X.toarray()

In [96]:
# Main (TF - IDF) =================================================
# Cargar txt y dividirlo por oraciones
path = "/content/Discurso.txt"
texto = cargar_texto(path)
dividir_por_oracion = oracion(texto)

# preprocesamiento de texto por oraciones
texto_preprocesado = []
for oracion in dividir_por_oracion:
  texto_preprocesado.append(preprocesar(oracion))

# List[List[str]] -> list[str]
lista_oraciones = []
for oracion in texto_preprocesado:
  lista_oraciones.append(' '.join(oracion))

# TF-IDF
lista_palabras, X = tf_idf(lista_oraciones)

# Media de score
filas, columnas = X.shape
score_media = []
for i in range(filas):
  elementos_diferentes_0 = 0
  suma_score = 0
  for j in range(columnas):
    if X[i, j] != 0:
      suma_score += X[i, j]
      elementos_diferentes_0 += 1
  if elementos_diferentes_0 > 0:
    score_media.append(suma_score/elementos_diferentes_0)
  else:
    score_media.append(0)

idx_doc_relevante = np.argsort(-np.array(score_media)) # obtener índices ordenados por relevancia
sublista = idx_doc_relevante[:70] # Número de índices arbitrario
sublista = np.sort(sublista) # Se ordena la sublista para evitar que se pierda el sentido del texto

# Generar resumen
resumen= ""
for indice in sublista:
  resumen += dividir_por_oracion[indice] + ". "

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /ro

In [97]:
resumen

'La verdad sea dicha, yo nunca me gradué. En realidad, esto es lo más cerca que jamás he estado de una graduación universitaria. Hoy les quiero contar tres historias de mi vida. Nada especial. Sólo tres historias. La primera historia versa sobre "conectar los puntos". Entonces, ¿por qué lo dejé?\n\nComenzó antes de que yo naciera. Y 17 años más tarde fui a la universidad. Después de seis meses, no le veía propósito alguno. Y me estaba gastando todos los ahorros que mis padres habían conseguido a lo largo de su vida. No era idílico. Me encantaba. Les daré un ejemplo. Nada de esto tenía ni la más mínima esperanza de aplicación práctica en mi vida. Y diseñamos el Mac con eso en su esencia. Fue el primer computador con tipografías bellas. Tienes que confiar en algo, tu instinto, el destino, la vida, el karma, lo que sea. Mi segunda historia es sobre el amor y la pérdida. Tuve suerte, supe pronto en mi vida qué era lo que más deseaba hacer. Woz y yo creamos Apple en el garaje de mis padres 